In [1]:
import pandas as pd
import numpy as np

import string
from re import sub

import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.preprocessing import LabelEncoder
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
from keras.layers import Input

In [3]:
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix


In [4]:
y_train, _, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, _, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))

In [5]:
y_train = y_train[slice(0, len(y_train), 3)]
x_train = x_train[slice(0, len(x_train), 3)]

In [6]:
y_test_np = []

for y in y_test:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_test_np.append(l)
    
y_test_np = np.asarray(y_test_np)
y_test_np.shape

(70000, 14)

In [7]:
y_train_np = []

for y in y_train:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_np.append(l)
    
y_train_np = np.asarray(y_train_np)
y_train_np.shape

(186667, 14)

In [8]:
del y_train
del y_test

import gc
gc.collect()

0

In [9]:
import tensorflow as tf

from time import time

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 6.1



In [10]:
y_train_np.shape

(186667, 14)

In [11]:
x_train.shape

(186667, 1000)

In [12]:
# del x_train

In [14]:
callback = EarlyStopping(monitor='val_loss',patience=2, min_delta=0.0001)

checkpoint = ModelCheckpoint("best_model_conv", monitor='loss', verbose=1, save_best_only=True, mode='min')

reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [15]:
model_l1 = Sequential()
# model_l1.add(Embedding(56000, 1000))
model_l1.add(Input(shape=(1000,1), name='input'))
model_l1.add(Conv1D(128, 5, activation='relu'))
model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train, y_train_np, epochs=10, verbose=1, validation_split=0.2, batch_size=1
                   ,callbacks=[reduce,callback,checkpoint])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 996, 128)          768       
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 14)                462       
                                                        

INFO:tensorflow:Assets written to: best_model_conv\assets


INFO:tensorflow:Assets written to: best_model_conv\assets


149333/149333 [==============================] - 2163s 14ms/step - loss: 1.4720 - accuracy: 0.4873 - val_loss: 22.4049 - val_accuracy: 0.0449 - lr: 0.0010
Epoch 2/10
149331/149333 [============================>.] - ETA: 0s - loss: 1.1927 - accuracy: 0.6025
Epoch 2: loss improved from 1.47201 to 1.19265, saving model to best_model_conv


INFO:tensorflow:Assets written to: best_model_conv\assets


...

INFO:tensorflow:Assets written to: best_model_conv\assets


149333/149333 [==============================] - 932s 6ms/step - loss: 1.1297 - accuracy: 0.6275 - val_loss: 23.1803 - val_accuracy: 0.0598 - lr: 0.0010
Epoch 4/10
149330/149333 [============================>.] - ETA: 0s - loss: 1.0993 - accuracy: 0.6402
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: loss improved from 1.12966 to 1.09929, saving model to best_model_conv


INFO:tensorflow:Assets written to: best_model_conv\assets


INFO:tensorflow:Assets written to: best_model_conv\assets


149333/149333 [==============================] - 964s 6ms/step - loss: 1.0993 - accuracy: 0.6402 - val_loss: 25.6297 - val_accuracy: 0.0989 - lr: 0.0010


In [16]:
import sys
import gc
gc.collect()

22562

In [17]:
score_l1 = model_l1.evaluate(x_test, y_test_np, verbose=1)
print('Test score:', score_l1[0])
print('Test accuracy:', score_l1[1])

2188/2188 [==============================] - 12s 5ms/step - loss: 5.9867 - accuracy: 0.5490
Test score: 5.986660480499268
Test accuracy: 0.5490285754203796


In [18]:
y_pred = model_l1.predict(x_test)

2188/2188 [==============================] - 7s 3ms/step


In [19]:
confusion_matrix(y_test_np, y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [23]:
classification_report(y_test_np, y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [20]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test_np, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2367  289  368  204  449  341  872   15   21   37   16   21    0    0]
 [ 154 3343  236  138  453   70  539   33   20    8    3    3    0    0]
 [ 286  343 2235  412  747  176  589   69   12   44    4   83    0    0]
 [ 116  143  423 3176  498   59  386   70   19  105    2    3    0    0]
 [ 174  313  347  278 3012  224  530   53   10   42   10    7    0    0]
 [ 405  221  303  291  475 2615  605   31    7   39    3    5    0    0]
 [ 128  264  135  100  257  140 3821   87   23   32    6    7    0    0]
 [  39  159  215  182  145   18  634 3494   35   67   11    1    0    0]
 [  14   61   59   65   40   11  153  127 4455   14    1    0    0    0]
 [  19   42   83  130   75   10  186   67    6 4015  363    4    0    0]
 [  21   34   31   48   48    0   63   29    2  549 4175    0    0    0]
 [ 277  275 1300  323  373  183  440   58    6   40    1 1724    0    0]
 [ 348  592 1491  364  720  378  772  145   54  106    3   27    0    0]
 [ 766  357 1240  324  766  730  692   32   25   38